# Introducción a la API Keras functional

## Autor

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
2. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 


## References

1. [Documentación de Keras](https://keras.io/getting-started/sequential-model-guide/)

# Introducción

La API funcional de Keras se guía por los siguientes dos conceptos:

1. Una capa es una instancia que acepta un tensor como argumento. La salida de una capa es otro tensor. Para construir un modelo, las instancias de capa son objetos que están encadenados entre sí a través de los tensores de entrada y salida. Esto tendrá un resultado final similar al apilamiento de varias capas en el modelo secuencial. Sin embargo, el uso de instancias de capa facilita que los modelos tengan entradas y salidas auxiliares o múltiples, ya que la entrada / salida de cada capa será fácilmente accesible.

2. Un modelo es una función entre uno o más tensores de entrada y tensores de salida. Entre la entrada y la salida del modelo, los tensores son las instancias de capa que están encadenados entre sí por los tensores de entrada y salida de la capa. Un modelo es, por lo tanto, una función de una o más capas de entrada y una o más capas de salida. La instancia del modelo formaliza el gráfico computacional sobre cómo fluyen los datos de entrada (s) a salida (s).


Para ilustrar, la API funcional, una capa convolucional bidimensional, Conv2D, con 32 filtros y con *x* como tensor de entrada de capa, y como tensor de salida de capa se puede escribir como:

y = Conv2D(32)(x)

In [1]:
import numpy as np
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [4]:
# load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [5]:
# from sparse label to categorical
num_labels = len(np.unique(y_train))
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [6]:
# reshape and normalize input images
image_size = x_train.shape[1]
x_train = np.reshape(x_train,[-1, image_size, image_size, 1])
x_test = np.reshape(x_test,[-1, image_size, image_size, 1])
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

In [7]:
# network parameters
input_shape = (image_size, image_size, 1)
batch_size = 128
kernel_size = 3
filters = 64
dropout = 0.3

In [9]:
# use functional API to build cnn layers
inputs = Input(shape=input_shape)
y = Conv2D(filters=filters,
           kernel_size=kernel_size,
           activation='relu')(inputs)
y = MaxPooling2D()(y) # pool_size=2, por defecto
y = Conv2D(filters=filters,
           kernel_size=kernel_size,
           activation='relu')(y)
y = MaxPooling2D()(y)
y = Conv2D(filters=filters,
           kernel_size=kernel_size,
           activation='relu')(y)
# image to vector before connecting to dense layer
y = Flatten()(y)
# dropout regularization
y = Dropout(dropout)(y)
outputs = Dense(num_labels, activation='softmax')(y)
# build the model by supplying inputs/outputs
model = Model(inputs=inputs, outputs=outputs)

In [ ]:
y = inputs
for i in range(3):
    y = Conv2D(filters=filters,
           kernel_size=kernel_size,
           activation='relu')(y)
    y = MaxPooling2D()(y) # pool_size=2, por defecto
    filters *=2

In [10]:
# network model in text
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 11, 64)        36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0   

In [11]:
# classifier loss, Adam optimizer, classifier accuracy
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
# validation_size=0.2
# validation_split=validation_size
#

In [9]:
# train the model with input images and labels
model.fit(x_train,
          y_train,
          validation_data=(x_test, y_test),
          epochs=20, 
          batch_size=batch_size)

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 85s 1ms/sample - loss: 0.2740 - accuracy: 0.9135 - val_loss: 0.0565 - val_accuracy: 0.9816
Epoch 2/20
60000/60000 [==============================] - 83s 1ms/sample - loss: 0.0721 - accuracy: 0.9780 - val_loss: 0.0369 - val_accuracy: 0.9882
Epoch 3/20
60000/60000 [==============================] - 89s 1ms/sample - loss: 0.0525 - accuracy: 0.9838 - val_loss: 0.0322 - val_accuracy: 0.9894
Epoch 4/20
60000/60000 [==============================] - 95s 2ms/sample - loss: 0.0422 - accuracy: 0.9868 - val_loss: 0.0274 - val_accuracy: 0.9904
Epoch 5/20
60000/60000 [==============================] - 91s 2ms/sample - loss: 0.0363 - accuracy: 0.9879 - val_loss: 0.0259 - val_accuracy: 0.9915
Epoch 6/20
60000/60000 [==============================] - 89s 1ms/sample - loss: 0.0305 - accuracy: 0.9906 - val_loss: 0.0272 - val_accuracy: 0.9908
Epoch 7/20
60000/60000 [==============================] 

In [10]:
# model accuracy on test dataset
score = model.evaluate(x_test,
                       y_test,
                       batch_size=batch_size,
                       verbose=0)
print("\nTest accuracy: %.1f%%" % (100.0 * score[1]))


Test accuracy: 99.4%


# Modelo con dos ramas

## import

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, Conv2D
from tensorflow.keras.layers import MaxPooling2D, Flatten, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical, plot_model

## lectura de datos

In [ ]:
(x_train,y_train), (x_test,y_test) = mnist.load()

## Transforma labels

In [ ]:
num_labels = len(unique(y_train))
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

## Adecua los datos de entrda: reshape

In [ ]:
image_size = x_train.shape[1]
x_train.reshape([-1,image_size,image_size,1])
x_test.reshape([-1,image_size,image_size,1])

x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255

input_shape = (image_size, image_size,1)

## Parámetros de la red

In [ ]:
# net parameters
batch_size = 32
epochs = 20
kernel_size = 3
n_filters = 32
dropout = 0.4

## Diseña las dos ramas

In [ ]:
#left branch
left_inputs = Input(shape=input_shape)
x = left_inputs
filters = n_filters

for i in range(3):
    x = Conv2D(filters= filters,
              kernel_size=kernel_size,
              padding='same',
              activation='relu')(x)
    x = Dropout(dropout)(x)
    x = MaxPooling(x,2)(x)
    filters*=2

#right branch
right_inputs = Input(shape=input_shape)
y = right_inputs
filters = n_filters

for i in range(3):
    y = Conv2D(filters= filters,
              kernel_size=kernel_size,
              padding='same',
              activation='relu'
              dilation=2)(y) # defect =1
    y = Dropout(dropout)(y)
    y = MaxPooling(x,2)(y)# defect=2
    filters*=2
    

## Mezcla las dos ramas

In [ ]:
# merge the branches
y = concatenate([x,y])
#
y = Flattent()(y)
y= Dropout(dropot)(y)
outputs = Dense(num_labels, activation='softmax')(y)

# Crea el modelo

In [ ]:
# build
model = Model(inputs, outputs)
#summary and plot
model.summary()
plot_model(model'cnn_model.png')

## Compila

In [ ]:
# compile
model.compile(loss='categorical_crossentropy',
             optimize ='adam',
             metrics=[accuracy])

## Entrenamiento

In [ ]:
history=model.fit([x_train],
                   y_train,
                  validation_data= ([x_test],y_test)
                  epochs= epochs, 
                  batch_size=batch_size)

## Evaluación

In [ ]:
score = model.evaluate([x_test],y_test, 
                       batch_size=batch_size,
                       verbose =0, )

print("\nTest accuracy: %.1f%%" % (100.0 * score[1]))

En el modelo ambas ramas tienen el mismo tamaño de kernel de 3, la rama derecha usa una tasa de dilatación de 2. 


La figura muestra el efecto de diferentes tasas de dilatación en un kernel con tamaño 3. 

<figure>
<center>
<img src="./Imagenes/dilation.png" width="800" height="600" align="center"/>
</center>
<figcaption>
<p style="text-align:center">Dilatación de kernels</p>
</figcaption>
</figure>

La idea es que al aumentar el tamaño de receptividad efectiva  del kernel utilizando la tasa de dilatación, la CNN permitirá a la rama correcta aprender diferentes mapas de características. El uso de una tasa de dilatación mayor que 1 es un método aproximado computacionalmente eficiente para aumentar el tamaño del campo receptivo. Es aproximado ya que el núcleo no es en realidad un núcleo completo. Es eficiente ya que utilizamos el mismo número de operaciones que con una tasa de dilatación igual a 1.

Para apreciar el concepto del campo receptivo, observe que cuando el núcleo calcula cada punto de un mapa de características (features), su entrada es un parche en el mapa de características de la capa anterior que también depende de su mapa de características de la capa anterior. Si continuamos rastreando esta dependencia hasta la imagen de entrada, el núcleo depende de un parche de imagen llamado campo receptivo.
 
Usaremos la opción *padding = 'same'* para asegurarnos de que no tendremos dimensiones negativas del tensor cuando se use el CNN dilatado. Al usar *padding = 'same'*, mantendremos las dimensiones de la entrada igual que los mapas de características de salida. Esto se logra rellenando la entrada con ceros para asegurarse de que la salida tenga el mismo tamaño.